In [11]:
import pandas as pd

In [64]:
import string
printable = set(string.printable)

def cleanup(x):
    x = " ".join(x.split("\\n"))
    x = " ".join(x.split("\\t"))
    x = " ".join(x.split("\\r"))
    x = " ".join(x.split("\n"))
    x = " ".join(x.split("\t"))
    x = " ".join(x.split("\r"))
    x = " ".join(x.split(","))
    x = " ".join(x.split("."))
    x = " ".join(x.split("!"))
    x = " ".join(x.split("?"))
    x = x.lower()
    x = "".join(list(filter(lambda c: c in printable, x)))
    x = " ".join(filter(lambda z: z != '', x.split(" ")))
    return x

In [71]:
df = pd.read_csv("toxic-train.csv")
df['comment_text'] = df['comment_text'].apply(cleanup)

In [66]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

In [67]:
sentScores = df['comment_text'].apply(analyser.polarity_scores)

In [68]:
for k in list(sentScores[0].keys()):
    df[k] = [ d[k] for d in sentScores ]

In [69]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,neg,neu,pos,compound
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0,0.000,0.905,0.095,0.5574
1,000103f0d9cfb60f,d'aww he matches this background colour i'm se...,0,0,0,0,0,0,0.101,0.754,0.146,0.2263
2,000113f07ec002fd,hey man i'm really not trying to edit war it's...,0,0,0,0,0,0,0.083,0.849,0.068,-0.1779
3,0001b41b1c6bb37e,""" more i can't make any real suggestions on im...",0,0,0,0,0,0,0.022,0.916,0.062,0.5106
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,0,0,0,0,0,0.000,0.663,0.337,0.6808


In [70]:
df.to_csv("toxic-train_vader.csv")

In [43]:
import nltk
import re, string
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer

def remove_noise(tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [45]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [109]:
df['lemmas'] = [ text.split(" ") for text in df['comment_text'] ]

In [110]:
df['lemmas'] = df['lemmas'].apply(remove_noise)

In [111]:
df.to_csv("toxic-train_w_lem_NLTK.csv")

In [113]:
nontoxic_tokens_list = df[df['toxic']==0]['lemmas'].values
toxic_tokens_list = df[df['toxic']==1]['lemmas'].values

In [131]:
def get_tokens_for_model(cleaned_tokens_list):
    for tokens_list in cleaned_tokens_list:
        yield dict([token, True] for token in tokens_list)

nontoxic_tokens_for_model = get_tokens_for_model(nontoxic_tokens_list)
toxic_tokens_for_model = get_tokens_for_model(toxic_tokens_list)

In [132]:
import random

nontoxic_dataset = [(dict, "Nontoxic") for dict in nontoxic_tokens_for_model]

toxic_dataset = [(dict, "Toxic") for dict in toxic_tokens_for_model]

dataset = nontoxic_dataset + toxic_dataset

random.shuffle(dataset)

train_data = dataset[7000:]
test_data = dataset[:7000]

In [133]:
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

Accuracy is: 0.9454285714285714


In [134]:
from nltk.tokenize import word_tokenize

custom_text = ### Censored ###

custom_tokens = remove_noise(custom_text.split(" "))

print(classifier.classify(dict([token, True] for token in custom_tokens)))

Toxic


In [136]:
from nltk.tokenize import word_tokenize

custom_text = "I love you!"

custom_tokens = remove_noise(custom_text.split(" "))

print(classifier.classify(dict([token, True] for token in custom_tokens)))

Nontoxic
